# Machine learning report - Credit card fraud detection

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

training_file_path = 'training.csv'
X_train = pd.read_csv(training_file_path)

test_file_path = 'test.csv'
X_test = pd.read_csv(test_file_path)

## 1. Manipulation des données
### 1.1. Données déséquilibrées
Nous sommes ici face a un dataset en fort déséquilibre. En effet, le nombre de transactions frauduleuses est très faible par rapport au nombre de transactions normales (~0.2%). Nous allons donc devoir faire attention à cela lors de la modélisation.

Deux solutions s'offrent à nous:
- Effectuer un oversampling de la classe minoritaire (fraud) afin d'obtenir un dataset équilibré
- Effectuer un undersampling de la classe majoritaire (normal) afin d'obtenir un dataset équilibré

Nous allons ici choisir la première solution, en utilisant la methode resample de la librairie sklearn. Celle ci va nous permettre de créer un dataset équilibré en augmentant le nombre d'observations de la classe minoritaire.


In [50]:
X_train_no_fraud = X_train[X_train['FraudResult']==0]
X_train_fraud = X_train[X_train['FraudResult']==1]

# We calculate the percentage of the minority class
percentage_minority = len(X_train_fraud)/(len(X_train_no_fraud) + len(X_train_fraud)) * 100
print(f"Fraud percantage before upsampling: {round(percentage_minority, 2)}%")

X_train_fraud_upsampled = resample(X_train_fraud, replace=True, n_samples=len(X_train_no_fraud), random_state=123)

X_train_upsampled = pd.concat([X_train_no_fraud, X_train_fraud])

y = X_train_upsampled['FraudResult']

Fraud percantage before upsampling: 0.2%


### 1.2. Premier tri et formatage des données
Nous allons ici effectuer un premier tri sur nos données. En effet, nous n'allons pas conserver les colonnes qui ne nous semblent pas pertinentes pour notre modele. Nous allons également formater les colonnes contenants des chaines de caractères afin de les rendre exploitables.

Nous eliminerons donc les colonnes suivantes:
- **BatchId**: Il s'agit de l'identifiant du lot auquel appartient la transaction. Cet identifiant ne nous semble pas pertinent pour détecter une fraude.
- **CurrencyCode**: Il s'agit du code de la devise utilisée pour la transaction. Cet identifiant ne nous est pas utile car nous avons ici affaire à une seule devise (UGX).
- **CountryCode**: Il s'agit du code du pays dans lequel la transaction a été effectuée. Cet identifiant ne nous est pas utile car nous avons ici affaire à un seul pays (256).
- **TransactionStartTime**: Il s'agit de la date et de l'heure de la transaction.

Nous retirons ensuite la partie textuelle des différents identifiants afin de ne garder que la partie numérique. Nous effectuons également un one hot encoding sur la colonne **ProductCategory** afin de la rendre exploitable par notre modèle.

In [51]:
# On print ici les valeurs unique de la variable currency code pour voir si il est pertinent de la garder.
print(X_train_upsampled['CurrencyCode'].unique())

# On indique ici les features que l'on veut garder
features = ['TransactionId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ChannelId', 'Amount', 'Value', 'PricingStrategy', 'ProductCategory']

# On applique ici la sélection des features
X_train_upsampled = X_train_upsampled[features]
X_test = X_test[features]

# On liste les collones d'identifiants auquel on veut enlever la partie textuelle
id_columns = ['TransactionId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ChannelId']

# On définit une fonction qui va enlever la partie textuelle des identifiants
def transform_id(feature):
    if isinstance(feature, str):
        return feature.split("_")[-1]
    else:
        return feature
    
# On applique la fonction sur les colonnes d'identifiants
for column in id_columns:
    X_train_upsampled[column] = X_train_upsampled[column].apply(transform_id)
    X_test[column] = X_test[column].apply(transform_id)

# On transforme les colonnes d'identifiants en int
X_train_upsampled[id_columns] = X_train_upsampled[id_columns].astype('int64')
X_test[id_columns] = X_test[id_columns].astype('int64')

# on print le type des colonnes pour vérifier que tout est ok
print(X_train_upsampled.dtypes)

# On applique du one hot encoding sur ProductCategory
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train_upsampled[['ProductCategory']]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[['ProductCategory']]))

# On remet les index
OH_cols_train.index = X_train_upsampled.index
OH_cols_test.index = X_test.index

# On enlève la colonne ProductCategory
num_X_train = X_train_upsampled.drop('ProductCategory', axis=1)
num_X_test = X_test.drop('ProductCategory', axis=1)

# On ajoute les colonnes encodées
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# On transforme les noms de colonnes en string
OH_X_train.columns = OH_X_train.columns.map(str)
OH_X_test.columns = OH_X_test.columns.map(str)

# On print les premières lignes du dataset pour voir si les identifiants ont bien été transformés
print(OH_X_train.head())

['UGX']
TransactionId        int64
AccountId            int64
SubscriptionId       int64
CustomerId           int64
ProviderId           int64
ProductId            int64
ChannelId            int64
Amount             float64
Value                int64
PricingStrategy      int64
ProductCategory     object
dtype: object
   TransactionId  AccountId  SubscriptionId  CustomerId  ProviderId  \
0          76871       3957             887        4406           6   
1          73770       4841            3829        4406           4   
2          26203       4229             222        4683           6   
3            380        648            2185         988           1   
4          28195       4841            3829         988           4   

   ProductId  ChannelId   Amount  Value  PricingStrategy    0    1    2    3  \
0         10          3   1000.0   1000                2  1.0  0.0  0.0  0.0   
1          6          2    -20.0     20                2  0.0  0.0  1.0  0.0   
2          1  

c:\Users\qjado\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 2. Modélisation
### 2.1 Choix du modèle
Nous allons ici utiliser un modèle de type **Random Forest**. Ce modèle est un modèle d'apprentissage supervisé qui peut être utilisé pour la classification ou la régression. Il s'agit d'un modèle très utilisé en machine learning car il est très performant et qu'il permet de traiter des données manquantes ou des données non équilibrées.

In [52]:
card_fraud_model = RandomForestClassifier(n_estimators=100, random_state=0)


card_fraud_model.fit(OH_X_train, y)

RandomForestClassifier(random_state=0)

We now make our predictions

In [53]:
# we make predictions which we will submit to Zindi, it need to be submit as a csv file containing the TransactionId and FraudResult
# The Transaction id is writen as "TransactionId_XXXX" so we need to split it and take the last element
predictions = card_fraud_model.predict(OH_X_test)
# we print the first 5 predictions
print(predictions[:5])

# We create a dataframe containing the TransactionId and the predictions
submission = pd.DataFrame({'TransactionId': OH_X_test.TransactionId, 'FraudResult': predictions})
print(submission.head())

# We add 'TransactionId_' to the TransactionId column
submission['TransactionId'] = 'TransactionId_' + submission['TransactionId'].astype(str)

# We save the dataframe as a csv file
submission.to_csv('submission.csv', index=False)

submission.head()

[0 0 0 0 0]
   TransactionId  FraudResult
0          50600            0
1          95109            0
2          47357            0
3          28185            0
4          22140            0


,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0
